In [30]:
# Importing necessary libraries
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [10]:
df = pd.read_csv('data/credit_risk1.csv')

In [12]:
df.head()

,borrower_id,income,employment_status,credit_score,debt_to_income_ratio,loan_amount_requested,age,education_level,marital_status,num_of_credit_inquiries,...,rental_payment_history,social_media_activity_score,spending_behavior_score,fraud_score,inconsistent_application_data,geo_location_mismatch,customer_lifetime_value,cross_product_ownership,customer_satisfaction_score,loan_default
0,1,57450.71230,Self-employed,342,0.311864,30208.015810,69,Bachelor's,Divorced,7,...,0.722975,62.439276,16.737285,0.901663,0,0,20804.027260,2,4.656849,0
1,2,47926.03548,Retired,684,0.196757,22258.352220,67,High School,Single,9,...,0.531154,15.184345,45.247005,0.805699,0,0,45482.425960,1,30.595614,0
2,3,59715.32807,Employed,734,0.300241,9055.870925,60,Master's,Married,8,...,0.730591,4.676001,51.078299,0.071612,0,0,26803.323080,1,67.577085,0
3,4,72845.44785,Retired,707,0.371848,6712.570653,46,High School,Single,0,...,0.094324,53.810566,53.170900,0.248914,0,0,28742.552550,3,13.516381,0
4,5,46487.69938,Self-employed,782,0.130496,27788.088420,38,PhD,Married,8,...,0.528656,23.489017,81.990043,0.819072,0,0,9635.409069,4,29.997218,0


In [13]:
# Fill missing values for numerical variables with the mean
df.fillna(df.select_dtypes(include='number').mean(), inplace=True)

# Fill missing values for categorical variables with the mode
for column in df.select_dtypes(include='object').columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [19]:
# Step 1: Check for categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(f"Categorical Columns: {categorical_cols}")

# Step 2: Label Encoding for columns with ordinal relationship or a small number of categories
# You can select specific columns for label encoding
label_enc = LabelEncoder()

# Apply label encoding to all categorical columns (if appropriate)
for col in categorical_cols:
    df[col] = label_enc.fit_transform(df[col])

# Step 3: One-Hot Encoding for categorical columns without an ordinal relationship
# This method is usually preferred for columns with no natural ordering

# Convert the categorical columns to one-hot encoding
data_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

Categorical Columns: Index(['employment_status', 'education_level', 'marital_status', 'loan_type',
       'repayment_schedule'],
      dtype='object')


In [20]:
# Features and target
X = df.drop(columns=["borrower_id", "loan_id", "loan_default"])  # Drop ID columns and the target column
y = df["loan_default"]

In [21]:
# Standardize the dataset for models like SVM, KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [23]:
# Step 2: Train and Evaluate 5 Models

# 1. Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)
log_reg_auc = roc_auc_score(y_test, log_reg.predict_proba(X_test)[:, 1])
print(f"Logistic Regression ROC-AUC: {log_reg_auc}")

Logistic Regression ROC-AUC: 0.5309951861676


In [24]:
# 2. Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
rf_clf_auc = roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:, 1])
print(f"Random Forest ROC-AUC: {rf_clf_auc}")

Random Forest ROC-AUC: 0.4551527654975931


In [25]:
# 3. Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
gb_clf_auc = roc_auc_score(y_test, gb_clf.predict_proba(X_test)[:, 1])
print(f"Gradient Boosting ROC-AUC: {gb_clf_auc}")

Gradient Boosting ROC-AUC: 0.43599567737498773


In [26]:
# 4. Support Vector Classifier
svc_clf = SVC(probability=True, random_state=42)
svc_clf.fit(X_train, y_train)
y_pred_svc = svc_clf.predict(X_test)
svc_clf_auc = roc_auc_score(y_test, svc_clf.predict_proba(X_test)[:, 1])
print(f"SVM ROC-AUC: {svc_clf_auc}")

SVM ROC-AUC: 0.49081442184890467


In [27]:
# 5. K-Nearest Neighbors Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
knn_clf_auc = roc_auc_score(y_test, knn_clf.predict_proba(X_test)[:, 1])
print(f"KNN ROC-AUC: {knn_clf_auc}")

KNN ROC-AUC: 0.5259357500736811


In [28]:
# Step 3: Model Selection based on ROC-AUC
model_scores = {
    "Logistic Regression": log_reg_auc,
    "Random Forest": rf_clf_auc,
    "Gradient Boosting": gb_clf_auc,
    "SVM": svc_clf_auc,
    "KNN": knn_clf_auc
}

best_model_name = max(model_scores, key=model_scores.get)
best_model_score = model_scores[best_model_name]

print(f"\nBest Model: {best_model_name} with ROC-AUC: {best_model_score}")


Best Model: Logistic Regression with ROC-AUC: 0.5309951861676


In [29]:
# Step 4: Use the Best Model for Final Predictions
if best_model_name == "Logistic Regression":
    best_model = log_reg
elif best_model_name == "Random Forest":
    best_model = rf_clf
elif best_model_name == "Gradient Boosting":
    best_model = gb_clf
elif best_model_name == "SVM":
    best_model = svc_clf
elif best_model_name == "KNN":
    best_model = knn_clf

# Final Prediction on Test Data using the Best Model
final_predictions = best_model.predict(X_test)

# Print final predictions
print("Final Predictions:", final_predictions)

Final Predictions: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [31]:
# Step 1: Create a directory to store the model if it doesn't exist
model_directory = "model"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

In [32]:
# Step 2: Save the best model to the folder
model_file_path = os.path.join(model_directory, "default_probability_model.pkl")
joblib.dump(best_model, model_file_path)
print(f"Best model saved to: {model_file_path}")

Best model saved to: model\default_probability_model.pkl
